In [15]:
import xlrd
import pickle

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns

from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec, mol2sentence
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [16]:
pickle_in = open("fold-0.pickle","rb")
load_dict =  pickle.load(pickle_in)

data_frame = load_dict['shuffled']

data_frame

,No.,FileHandle,SoluteName,Formula,Subset,Charge,Level1,Level2,Level3,Solvent,...,HP,OP,S,HS,OS,SP,SS,TotalArea,SoluteSMILES,SolventSMILES
1758,1759,0438pho,"diethyl2,4-dichlorophenylthiophosphate",H13C10O3P1S1CL2,[g],0,11,4,0,octanol,...,0.000,16.033,34.582,0.0,0.0,24.737,0.0,331.933,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,CCCCCCCCO
788,789,0071proa,propanal,H6C3O1,[g],0,2,4,1,diethylether,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,124.289,CCC=O,CCOCC
1726,1727,0210dic,"1,1-dichloro-2,2-difluoroethylmethylether",H4C3O1F2CL2,[g],0,9,1,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,178.993,CC1=CC(=O)CC(C1)(C)C,CCCCCCCCO
1939,1940,0110but,butylamine,H11C4N1,[g],0,3,1,0,secbutanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,164.243,CCCCN,CCC(C)O
1467,1468,0008noc,n-octane,H18C8,[g],0,2,1,1,methylformamide,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,243.846,CCCCCCCC,CNC=O
2110,2111,0021eth,ethene,H4C2,[a],0,2,1,4,water,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,87.417,C=C,O
739,740,0087pro,propanoicacid,H6C3O2,[g],0,2,5,0,dichloroethane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,135.630,CCC(=O)O,CC(Cl)Cl
1196,1197,0172Edi,"E-1,2-dichloroethene",H2C2CL2,[g],0,6,2,0,hexadecane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,126.855,C(=CCl)Cl,CCCCCCCCCCCCCCCC
111,112,0150mhy,m-hydroxybenzaldehyde,H6C7O2,[g],0,2,9,0,benzene,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,179.066,C1=CC(=CC(=C1)O)C=O,C1=CC=CC=C1
1820,1821,test4002,diiodomethane,H2C1I2,[g],0,15,0,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,136.531,C(I)I,CCCCCCCCO


In [17]:
# loc = ("MNSol_alldata.xlsx") 
# wb = xlrd.open_workbook(loc) 
# sheet = wb.sheet_by_index(0) 

# solutes = sheet.col_values(2)
# solvents = sheet.col_values(9)
# energies = sheet.col_values(10)

# solute_smis = []
# solvent_smis = []

# for solute in solutes[100:110]:
#     for compound in pcp.get_compounds(solute, 'name'):
#         solute_smis.append(compound.isomeric_smiles)
        
# for solvent in solvents[100:110]:
#     for compound in pcp.get_compounds(solvent, 'name'):
#         solvent_smis.append(compound.isomeric_smiles)

solute_smis = data_frame['SoluteSMILES'].tolist()
solvent_smis = data_frame['SolventSMILES'].tolist()
energies = data_frame['DeltaGsolv'].tolist()

print(len(solute_smis))
print(len(solvent_smis))
print(len(energies))

2530
2530
2530


In [18]:
solutesmis = [Chem.MolFromSmiles(x) for x in solute_smis]
solventsmis = [Chem.MolFromSmiles(x) for x in solvent_smis]

RDKit WARNING: [19:55:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:55:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:55:11] WARNING: not removing hydrogen atom without neighbors


In [19]:
solute_sentences = []
solvent_sentences = []

for solute_smi in solutesmis:
    solute_sentences.append(mol2alt_sentence(solute_smi, 1))
    
for solvent_smi in solventsmis:
    solvent_sentences.append(mol2alt_sentence(solvent_smi, 1))
    
print(solute_sentences[6])
print(solvent_sentences[6])

['2246728737', '3542456614', '2245384272', '2064788354', '2246699815', '4278941385', '864942730', '1510328189', '864662311', '1533864325']
['2246728737', '3537119515', '2245273601', '2406620359', '1016841875', '170898322', '1016841875', '170898322']


In [20]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_300dim.pkl')

solute_sequences = []
count = 0
sequences = []
solvent_sequences = []
Gsolv = []
l = len(solute_smis)

for i in range(l):
    flag = 0
    solute_substructures = []
    solvent_substructures = []
    
    for identifier in solute_sentences[i]:
        try:
            solute_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
    
    for identifier in solvent_sentences[i]:
        try:
            solvent_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            break
            
    if flag == 1:
        count += 1
        continue
    sequences.append((solute_substructures, solvent_substructures))
    solute_sequences.append(solute_substructures)
    solvent_sequences.append(solvent_substructures)
    Gsolv.append(energies[i])
    
    
print(count)
# solute_mol2vecs = [DfVec(x) for x in sentences2vec(solute_sentences, model, unseen='UNK')]
# solvent_mol2vecs = [DfVec(x) for x in sentences2vec(solvent_sentences, model, unseen='UNK')]

401


In [21]:
print(len(solute_sequences))
print(len(solvent_sequences))
print(len(Gsolv))
print(len(sequences))

# for i in solute_sequences:
#     print(np.asarray(i).shape)
# print('\n')
# for i in solvent_sequences:
#     print(np.asarray(i).shape)

2129
2129
2129
2129


In [22]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import math
import time
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer, LayerNorm

In [23]:
class TransformerModel(nn.Module):

    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0):
        super(TransformerModel, self).__init__()
        
        self.model_type = 'Transformer'
        self.solv_src_mask = None
        self.solu_src_mask = None
        self.ninp = ninp
        
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        
        self.solv_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.solu_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        self.fc1 = nn.Linear(600, 300)
        self.fc2 = nn.Linear(300, 1)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.fc1.bias.data.zero_()
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)

    def forward(self, solv_src, solu_src):
        if self.solv_src_mask is None or self.solv_src_mask.size(0) != len(solv_src):
            self.solv_src_mask = self._generate_square_subsequent_mask(len(solv_src))
            
        if self.solu_src_mask is None or self.solu_src_mask.size(0) != len(solu_src):
            self.solu_src_mask = self._generate_square_subsequent_mask(len(solu_src))
        
#         solv_src = self.pos_encoder(solv_src)
#         solu_src = self.pos_encoder(solu_src)
        
        solv_output = self.solv_encoder(solv_src, self.solv_src_mask)
        solu_output = self.solu_encoder(solu_src, self.solu_src_mask)
        
        solv_output = torch.squeeze(solv_output, 1)
        solu_output = torch.squeeze(solu_output, 1)
        
        inpu = torch.sum(solv_output, dim=0)
        inpv = torch.sum(solu_output, dim=0)
        
        inp = torch.cat((inpu, inpv), 0)
        
        #mlp
        x = F.relu(self.fc1(inp))
        output = self.fc2(x)
        
#         print(output.shape)
#         print(output)
        
        return output

In [24]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [25]:
ninp = 300 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 3 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 3 # the number of heads in the multiheadattention models
dropout = 0 # the dropout value

In [26]:
model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

# PRINTING MODEL & PARAMETERS 
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (solv_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=200, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=200, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=200, bias=True)
 

In [39]:
learning_rate = 0.00001
nest_momentum = 0.009

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [40]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

num_epochs = 100
splits = 10

best_model = model
best_loss = float("inf")
losses = []

# Training and 10 fold cross validation
kf = KFold(n_splits = 10)
kf.get_n_splits(sequences)

for train_index, test_index in kf.split(sequences):
    model = TransformerModel(ninp, nhead, nhid, nlayers, dropout)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
    
    seq_tr_va, seq_test = np.asarray(sequences)[train_index], np.asarray(sequences)[test_index]
    E_tr_va, E_test = np.asarray(Gsolv)[train_index], np.asarray(Gsolv)[test_index]
    seq_train, seq_val, E_train, E_val = train_test_split(seq_tr_va, E_tr_va, test_size=0.11)
    
    print("TRAIN:", len(seq_train), "VALIDATION:", len(seq_val), "TEST:", len(seq_test))
    
    #Training
    for epoch in range(num_epochs):   
        model.train()
        for i in range(len(seq_train)):
            solute, solvent = seq_train[i]
            energy = torch.FloatTensor([E_train[i]])
        
            # Forward pass
            solvent = torch.FloatTensor(solvent)
            solute = torch.FloatTensor(solute)
            
            solvent = torch.unsqueeze(solvent, 1)
            solute = torch.unsqueeze(solute, 1)
            
            output = model(solvent, solute)
            print(output.shape)
            loss = criterion(output, energy)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            
        #Validation
        eval_model.eval()
        outputs = []

        with torch.no_grad():
            for i in range(len(seq_val)):
                solute, solvent = seq_val[i]
                energy = torch.FloatTensor([E_val[i]])
        
                output = model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
                outputs.append(output)
                
            val_loss = criterion(torch.squeeze(torch.FloatTensor(E_val), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
            print('Validation MSE: {}'.format(val_loss))
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model
                
            scheduler.step()
            
    #Testing
    eval_model.eval()
    outputs = []

    with torch.no_grad():
        for i in range(len(seq_test)):
            solute, solvent = seq_test[i]
            energy = torch.FloatTensor([E_test[i]])
        
            output = best_model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
            outputs.append(output)
                
        test_loss = criterion(torch.squeeze(torch.FloatTensor(E_test), axis=0), torch.squeeze(torch.FloatTensor([outputs]), axis=0))
        print('Test MSE: {}'.format(test_loss))
        losses.append(test_loss)
    

TRAIN: 1705 VALIDATION: 211 TEST: 213
torch.Size([1])
Epoch [1/100], Loss: 273.8858
torch.Size([1])
Epoch [1/100], Loss: 970.6582
torch.Size([1])
Epoch [1/100], Loss: 1324.9637
torch.Size([1])
Epoch [1/100], Loss: 3756.2483
torch.Size([1])
Epoch [1/100], Loss: 2394.8853
torch.Size([1])
Epoch [1/100], Loss: 52.3952
torch.Size([1])
Epoch [1/100], Loss: 1.2443
torch.Size([1])
Epoch [1/100], Loss: 2950.1780
torch.Size([1])
Epoch [1/100], Loss: 3907.4802
torch.Size([1])
Epoch [1/100], Loss: 2995.8735
torch.Size([1])
Epoch [1/100], Loss: 643.1447
torch.Size([1])
Epoch [1/100], Loss: 553.8522
torch.Size([1])
Epoch [1/100], Loss: 28.1581
torch.Size([1])
Epoch [1/100], Loss: 98.9553
torch.Size([1])
Epoch [1/100], Loss: 17.0301
torch.Size([1])
Epoch [1/100], Loss: 0.0585
torch.Size([1])
Epoch [1/100], Loss: 138.8641
torch.Size([1])
Epoch [1/100], Loss: 89.7163
torch.Size([1])
Epoch [1/100], Loss: 55.7139
torch.Size([1])
Epoch [1/100], Loss: 7.0286
torch.Size([1])
Epoch [1/100], Loss: 15.4681
tor

Epoch [1/100], Loss: 1.8022
torch.Size([1])
Epoch [1/100], Loss: 2.8105
torch.Size([1])
Epoch [1/100], Loss: 4.4960
torch.Size([1])
Epoch [1/100], Loss: 9.4655
torch.Size([1])
Epoch [1/100], Loss: 0.7945
torch.Size([1])
Epoch [1/100], Loss: 0.8746
torch.Size([1])
Epoch [1/100], Loss: 3.8592
torch.Size([1])
Epoch [1/100], Loss: 0.0292
torch.Size([1])
Epoch [1/100], Loss: 0.0391
torch.Size([1])
Epoch [1/100], Loss: 7.5484
torch.Size([1])
Epoch [1/100], Loss: 0.0904
torch.Size([1])
Epoch [1/100], Loss: 0.3670
torch.Size([1])
Epoch [1/100], Loss: 0.2971
torch.Size([1])
Epoch [1/100], Loss: 40.2088
torch.Size([1])
Epoch [1/100], Loss: 13.8878
torch.Size([1])
Epoch [1/100], Loss: 0.4661
torch.Size([1])
Epoch [1/100], Loss: 0.1851
torch.Size([1])
Epoch [1/100], Loss: 73.6802
torch.Size([1])
Epoch [1/100], Loss: 0.3989
torch.Size([1])
Epoch [1/100], Loss: 0.3461
torch.Size([1])
Epoch [1/100], Loss: 8.1131
torch.Size([1])
Epoch [1/100], Loss: 4.3185
torch.Size([1])
Epoch [1/100], Loss: 3.3484
t

torch.Size([1])
Epoch [1/100], Loss: 0.7569
torch.Size([1])
Epoch [1/100], Loss: 0.1104
torch.Size([1])
Epoch [1/100], Loss: 0.5462
torch.Size([1])
Epoch [1/100], Loss: 14.5008
torch.Size([1])
Epoch [1/100], Loss: 12.5079
torch.Size([1])
Epoch [1/100], Loss: 0.4995
torch.Size([1])
Epoch [1/100], Loss: 1.8364
torch.Size([1])
Epoch [1/100], Loss: 1.0896
torch.Size([1])
Epoch [1/100], Loss: 117.1798
torch.Size([1])
Epoch [1/100], Loss: 1.2128
torch.Size([1])
Epoch [1/100], Loss: 15.0998
torch.Size([1])
Epoch [1/100], Loss: 1.9658
torch.Size([1])
Epoch [1/100], Loss: 6.1662
torch.Size([1])
Epoch [1/100], Loss: 0.5431
torch.Size([1])
Epoch [1/100], Loss: 18.7949
torch.Size([1])
Epoch [1/100], Loss: 23.7283
torch.Size([1])
Epoch [1/100], Loss: 5.3610
torch.Size([1])
Epoch [1/100], Loss: 3.9340
torch.Size([1])
Epoch [1/100], Loss: 1.8249
torch.Size([1])
Epoch [1/100], Loss: 2.7465
torch.Size([1])
Epoch [1/100], Loss: 1.7110
torch.Size([1])
Epoch [1/100], Loss: 13.0490
torch.Size([1])
Epoch [1

torch.Size([1])
Epoch [1/100], Loss: 0.9324
torch.Size([1])
Epoch [1/100], Loss: 0.6485
torch.Size([1])
Epoch [1/100], Loss: 7.3233
torch.Size([1])
Epoch [1/100], Loss: 2.9685
torch.Size([1])
Epoch [1/100], Loss: 4.7484
torch.Size([1])
Epoch [1/100], Loss: 3.7755
torch.Size([1])
Epoch [1/100], Loss: 7.5356
torch.Size([1])
Epoch [1/100], Loss: 2.9463
torch.Size([1])
Epoch [1/100], Loss: 18.5831
torch.Size([1])
Epoch [1/100], Loss: 4.4657
torch.Size([1])
Epoch [1/100], Loss: 10.9721
torch.Size([1])
Epoch [1/100], Loss: 1.4678
torch.Size([1])
Epoch [1/100], Loss: 0.2328
torch.Size([1])
Epoch [1/100], Loss: 0.3623
torch.Size([1])
Epoch [1/100], Loss: 0.3600
torch.Size([1])
Epoch [1/100], Loss: 0.1740
torch.Size([1])
Epoch [1/100], Loss: 52.5150
torch.Size([1])
Epoch [1/100], Loss: 16.1881
torch.Size([1])
Epoch [1/100], Loss: 25.7573
torch.Size([1])
Epoch [1/100], Loss: 0.2551
torch.Size([1])
Epoch [1/100], Loss: 0.0008
torch.Size([1])
Epoch [1/100], Loss: 83.6031
torch.Size([1])
Epoch [1/1

Epoch [1/100], Loss: 1.4973
torch.Size([1])
Epoch [1/100], Loss: 23.2588
torch.Size([1])
Epoch [1/100], Loss: 0.0837
torch.Size([1])
Epoch [1/100], Loss: 0.0778
torch.Size([1])
Epoch [1/100], Loss: 5.8370
torch.Size([1])
Epoch [1/100], Loss: 11.9684
torch.Size([1])
Epoch [1/100], Loss: 2.3413
torch.Size([1])
Epoch [1/100], Loss: 0.0915
torch.Size([1])
Epoch [1/100], Loss: 0.6074
torch.Size([1])
Epoch [1/100], Loss: 5.7731
torch.Size([1])
Epoch [1/100], Loss: 2.9429
torch.Size([1])
Epoch [1/100], Loss: 2.6589
torch.Size([1])
Epoch [1/100], Loss: 1.4403
torch.Size([1])
Epoch [1/100], Loss: 1.3888
torch.Size([1])
Epoch [1/100], Loss: 0.5471
torch.Size([1])
Epoch [1/100], Loss: 1.3843
torch.Size([1])
Epoch [1/100], Loss: 0.6908
torch.Size([1])
Epoch [1/100], Loss: 39.5790
torch.Size([1])
Epoch [1/100], Loss: 1.0290
torch.Size([1])
Epoch [1/100], Loss: 0.8528
torch.Size([1])
Epoch [1/100], Loss: 0.5111
torch.Size([1])
Epoch [1/100], Loss: 0.6280
torch.Size([1])
Epoch [1/100], Loss: 8.8785
t

Epoch [1/100], Loss: 0.0258
torch.Size([1])
Epoch [1/100], Loss: 0.8647
torch.Size([1])
Epoch [1/100], Loss: 2.3380
torch.Size([1])
Epoch [1/100], Loss: 0.2963
torch.Size([1])
Epoch [1/100], Loss: 0.3085
torch.Size([1])
Epoch [1/100], Loss: 0.2226
torch.Size([1])
Epoch [1/100], Loss: 0.0080
torch.Size([1])
Epoch [1/100], Loss: 0.0004
torch.Size([1])
Epoch [1/100], Loss: 0.0539
torch.Size([1])
Epoch [1/100], Loss: 0.2899
torch.Size([1])
Epoch [1/100], Loss: 5.4712
torch.Size([1])
Epoch [1/100], Loss: 0.0544
torch.Size([1])
Epoch [1/100], Loss: 0.0067
torch.Size([1])
Epoch [1/100], Loss: 0.1727
torch.Size([1])
Epoch [1/100], Loss: 0.1898
torch.Size([1])
Epoch [1/100], Loss: 0.2858
torch.Size([1])
Epoch [1/100], Loss: 0.6317
torch.Size([1])
Epoch [1/100], Loss: 0.0675
torch.Size([1])
Epoch [1/100], Loss: 0.0098
torch.Size([1])
Epoch [1/100], Loss: 5.9949
torch.Size([1])
Epoch [1/100], Loss: 0.1518
torch.Size([1])
Epoch [1/100], Loss: 9.2419
torch.Size([1])
Epoch [1/100], Loss: 0.2727
torc

Epoch [1/100], Loss: 6.0040
torch.Size([1])
Epoch [1/100], Loss: 0.0686
torch.Size([1])
Epoch [1/100], Loss: 10.6224
torch.Size([1])
Epoch [1/100], Loss: 0.0269
torch.Size([1])
Epoch [1/100], Loss: 66.0847
torch.Size([1])
Epoch [1/100], Loss: 1.3198
torch.Size([1])
Epoch [1/100], Loss: 16.3392
torch.Size([1])
Epoch [1/100], Loss: 0.3718
torch.Size([1])
Epoch [1/100], Loss: 0.1000
torch.Size([1])
Epoch [1/100], Loss: 0.0876
torch.Size([1])
Epoch [1/100], Loss: 0.4720
torch.Size([1])
Epoch [1/100], Loss: 0.0333
torch.Size([1])
Epoch [1/100], Loss: 8.2016
torch.Size([1])
Epoch [1/100], Loss: 0.2175
torch.Size([1])
Epoch [1/100], Loss: 35.9887
torch.Size([1])
Epoch [1/100], Loss: 1.0134
torch.Size([1])
Epoch [1/100], Loss: 0.0871
torch.Size([1])
Epoch [1/100], Loss: 0.1765
torch.Size([1])
Epoch [1/100], Loss: 5.3267
torch.Size([1])
Epoch [1/100], Loss: 5.8418
torch.Size([1])
Epoch [1/100], Loss: 2.6440
torch.Size([1])
Epoch [1/100], Loss: 3.2933
torch.Size([1])
Epoch [1/100], Loss: 14.9364

Epoch [1/100], Loss: 0.1530
torch.Size([1])
Epoch [1/100], Loss: 2.7055
torch.Size([1])
Epoch [1/100], Loss: 0.6476
torch.Size([1])
Epoch [1/100], Loss: 2.7062
torch.Size([1])
Epoch [1/100], Loss: 1.3863
torch.Size([1])
Epoch [1/100], Loss: 0.6921
torch.Size([1])
Epoch [1/100], Loss: 0.1936
torch.Size([1])
Epoch [1/100], Loss: 0.0790
torch.Size([1])
Epoch [1/100], Loss: 3.7761
torch.Size([1])
Epoch [1/100], Loss: 2.9043
torch.Size([1])
Epoch [1/100], Loss: 0.3242
torch.Size([1])
Epoch [1/100], Loss: 1.0329
torch.Size([1])
Epoch [1/100], Loss: 3.5471
torch.Size([1])
Epoch [1/100], Loss: 0.0353
torch.Size([1])
Epoch [1/100], Loss: 0.3907
torch.Size([1])
Epoch [1/100], Loss: 1.0755
torch.Size([1])
Epoch [1/100], Loss: 0.3645
torch.Size([1])
Epoch [1/100], Loss: 0.0336
torch.Size([1])
Epoch [1/100], Loss: 4.3889
torch.Size([1])
Epoch [1/100], Loss: 0.1854
torch.Size([1])
Epoch [1/100], Loss: 0.3247
torch.Size([1])
Epoch [1/100], Loss: 0.8804
torch.Size([1])
Epoch [1/100], Loss: 0.9032
torc

Epoch [1/100], Loss: 3.7994
torch.Size([1])
Epoch [1/100], Loss: 0.9043
torch.Size([1])
Epoch [1/100], Loss: 0.4235
torch.Size([1])
Epoch [1/100], Loss: 0.0693
torch.Size([1])
Epoch [1/100], Loss: 8.9599
torch.Size([1])
Epoch [1/100], Loss: 4.9233
torch.Size([1])
Epoch [1/100], Loss: 2.7590
torch.Size([1])
Epoch [1/100], Loss: 2.2488
torch.Size([1])
Epoch [1/100], Loss: 0.1365
torch.Size([1])
Epoch [1/100], Loss: 0.3451
torch.Size([1])
Epoch [1/100], Loss: 0.0066
torch.Size([1])
Epoch [1/100], Loss: 0.1912
torch.Size([1])
Epoch [1/100], Loss: 0.2352
torch.Size([1])
Epoch [1/100], Loss: 2.9109
torch.Size([1])
Epoch [1/100], Loss: 10.6637
torch.Size([1])
Epoch [1/100], Loss: 2.7314
torch.Size([1])
Epoch [1/100], Loss: 0.0077
torch.Size([1])
Epoch [1/100], Loss: 0.2880
torch.Size([1])
Epoch [1/100], Loss: 0.2424
torch.Size([1])
Epoch [1/100], Loss: 0.7482
torch.Size([1])
Epoch [1/100], Loss: 0.4805
torch.Size([1])
Epoch [1/100], Loss: 0.8615
torch.Size([1])
Epoch [1/100], Loss: 2.5566
tor

NameError: name 'eval_model' is not defined

In [61]:
print(losses)

with open('losses.txt', 'a') as f:
    for item in losses:
        f.write("%s\n" % item)

[tensor([-3.1967]), tensor([-3.5724]), tensor([-3.4067]), tensor([-3.3865]), tensor([-3.7676])]
[-5.28, -6.88, -5.86, -6.17, -6.64]


In [ ]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')